In [79]:
# %load detect.py
import argparse
import os
import platform
import shutil
import time
from pathlib import Path
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:/Program Files/Tesseract-OCR/tesseract.exe"
from PIL import Image
import cv2
import torch
import torch.backends.cudnn as cudnn
from numpy import random

from utils.google_utils import attempt_load
from utils.datasets import LoadStreams, LoadImages
from utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_coords, xyxy2xywh, strip_optimizer)
from utils.plots import plot_one_box
from utils.torch_utils import select_device, load_classifier, time_synchronized

from models.models import *
from utils.datasets import *
from utils.general import *



def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

def detect(save_img=False):
    out, source, weights, view_img, save_txt, imgsz, cfg, names = \
        opt.output, opt.source, opt.weights, opt.view_img, opt.save_txt, opt.img_size, opt.cfg, opt.names
    webcam = source == '0' or source.startswith('rtsp') or source.startswith('http') or source.endswith('.txt')

    # Initialize
    device = select_device(opt.device)
    if os.path.exists(out):
        shutil.rmtree(out)  # delete output folder
    os.makedirs(out)  # make new output folder
    half = device.type != 'cpu'  # half precision only supported on CUDA

    # Load model
    model = Darknet(cfg, imgsz).cuda()
    try:
        model.load_state_dict(torch.load(weights[0], map_location=device)['model'])
        #model = attempt_load(weights, map_location=device)  # load FP32 model
        #imgsz = check_img_size(imgsz, s=model.stride.max())  # check img_size
    except:
        load_darknet_weights(model, weights[0])
    model.to(device).eval()
    if half:
        model.half()  # to FP16

    # Second-stage classifier
    classify = False
    if classify:
        modelc = load_classifier(name='resnet101', n=2)  # initialize
        modelc.load_state_dict(torch.load('weights/resnet101.pt', map_location=device)['model'])  # load weights
        modelc.to(device).eval()

    # Set Dataloader
    vid_path, vid_writer = None, None
    if webcam:
        view_img = True
        cudnn.benchmark = True  # set True to speed up constant image size inference
        dataset = LoadStreams(source, img_size=imgsz)
    else:
        save_img = True
        dataset = LoadImages(source, img_size=imgsz, auto_size=64)

    # Get names and colors
    names = load_classes(names)
    colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]

    strings = [] #多數決存放的string
    bestimg=0

    # Run inference
    t0 = time.time()
    img = torch.zeros((1, 3, imgsz, imgsz), device=device)  # init img
    _ = model(img.half() if half else img) if device.type != 'cpu' else None  # run once
    for path, img, im0s, vid_cap in dataset:
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)

        # Inference
        t1 = time_synchronized()
        pred = model(img, augment=opt.augment)[0]

        # Apply NMS
        pred = non_max_suppression(pred, opt.conf_thres, opt.iou_thres, classes=opt.classes, agnostic=opt.agnostic_nms)
        t2 = time_synchronized()

        # Apply Classifier
        if classify:
            pred = apply_classifier(pred, modelc, img, im0s)

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            if webcam:  # batch_size >= 1
                p, s, im0 = path[i], '%g: ' % i, im0s[i].copy()
            else:
                p, s, im0 = path, '', im0s
           
           
            

            save_path = str(Path(out) / Path(p).name)
            txt_path = str(Path(out) / Path(p).stem) + ('_%g' % dataset.frame if dataset.mode == 'video' else '')
            s += '%gx%g ' % img.shape[2:]  # print string
            gn = torch.tensor(im0.shape)[[1, 0, 1, 0]]  # normalization gain whwh
            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += '%g %ss, ' % (n, names[int(c)])  # add to string

                # Write results
                for *xyxy, conf, cls in det:
                    if save_txt:  # Write to file
                        xywh = (xyxy2xywh(torch.tensor(xyxy).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                        with open(txt_path + '.txt', 'a') as f:
                            f.write(('%g ' * 5 + '\n') % (cls, *xywh))  # label format
                    if save_img:# 裁剪檢測框區域
                        x1, y1, x2, y2 = map(int, xyxy)
                        cropped_img = im0[y1:y2, x1:x2]
                         # 儲存裁剪後的圖片
                        save_cropped_path = save_path.replace('.jpg', f'_{int(x1)}_{int(y1)}_{int(x2)}_{int(y2)}.jpg')
                        # cv2.imwrite(save_cropped_path, cropped_img)
                        gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                        _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                        denoised_img = cv2.fastNlMeansDenoising(binary_img, None, 10, 7, 21)
                        enhanced_img = cv2.equalizeHist(denoised_img)# enhanced_img = cv2.bitwise_not(enhanced_img)
                        pil_img = Image.fromarray(enhanced_img)  
                        text = pytesseract.image_to_string(pil_img, lang='eng')  # 英文文字檢測
                        allowed_chars = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                        text_filtered = ''.join(filter(allowed_chars.__contains__, text)).upper()
                        #print("Detected Text:", text_filtered)
                        cv2.putText(im0, text_filtered, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                    def calculate_formula_result(letters):
                        values = {
                                    'A': 10, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20,
                                    'K': 21, 'L': 23, 'M': 24, 'N': 25, 'O': 26, 'P': 27, 'Q': 28, 'R': 29, 'S': 30, 'T': 31,
                                    'U': 32, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, '1': 1,  '2':  2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9
                                }
                        total = 0
                        for i, letter in enumerate(letters):
                            if letter in values:
                                total += values[letter] * (2 ** i)
                        return total % 11  
                    


                    # path1 ="C:/Users/p9514/YOLOv4/data/output/SEKU5875349"
                    # countimg = 0
                    # for i in os.listdir(path1):
                    #     countimg = countimg+1  
                    
                    def find_most_common_char(string):   # 多數決
                        char_count = {}
                        for char in string:
                            if char in char_count:
                                char_count[char] += 1
                            else:
                                char_count[char] = 1

                        max_count = 0
                        most_common_char = ''

                        for char, count in char_count.items():
                            if count > max_count:
                                max_count = count
                                most_common_char = char
                                bestimg = max_count

                            # if bestimg >0 :
                            
                            #     print("績效為:",bestimg/countimg)
                            #     break
                        return most_common_char,max_count
                    
                    if len(text_filtered)>=10:
                        
                        input_letters = text_filtered
                        input_letters = input_letters[:10]
                        result = calculate_formula_result(input_letters)
                        # print("Detected Text:", input_letters)
                        # print(result,type(result))
                        # print(type(save_path[-5]))
                        if  result == int(save_path[-5]):
                            print("Detected Text:", input_letters)
                            print(result)
                            strings.append(input_letters)
                            
                # print(strings)
                    print("出現最多次數的貨號",find_most_common_char(strings))
                        
                        

                    


                
                
                    # if view_img:# 裁剪檢測框區域
                    #     x1, y1, x2, y2 = map(int, xyxy)
                    #     cropped_view = im0[y1:y2, x1:x2]
                    #      # 儲存裁剪後的圖片
                    #     save_cropped_path = save_path.replace('.mkv', f'_{int(x1)}_{int(y1)}_{int(x2)}_{int(y2)}.mkv')
                    #     cv2.imwrite(save_cropped_path, cropped_view)
                    #     gray_img = cv2.cvtColor(cropped_view, cv2.COLOR_BGR2GRAY)
                    #     _, binary_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
                    #     denoised_img = cv2.fastNlMeansDenoising(binary_img, None, 10, 7, 21)
                    #     enhanced_img = cv2.equalizeHist(denoised_img)# enhanced_img = cv2.bitwise_not(enhanced_img)
                    #     pil_img = Image.fromarray(enhanced_img)  
                    #     text = pytesseract.image_to_string(pil_img, lang='eng')  # 英文文字檢測
                    #     allowed_chars = set('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
                    #     text_filtered = ''.join(filter(allowed_chars.__contains__, text)).upper()
                    #     print("Detected Text:", text_filtered)
                    #     cv2.putText(im0, text_filtered, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
                if save_img or view_img:  # Add bbox to image
                    label = '%s %.2f' % (names[int(cls)], conf)
                    plot_one_box(xyxy, im0, color=colors[int(cls)], line_thickness=3)

            # Print time (inference + NMS)
            print('%sDone. (%.3fs)' % (s, t2 - t1))

            # Stream results
            if view_img:
                cv2.imshow(p, im0)
                if cv2.waitKey(1) == ord('q'):  # q to quit
                    raise StopIteration

            # Save results (image with detections)
            if save_img:
                if dataset.mode == 'images':
                    cv2.imwrite(save_path, im0)
                else:
                    if vid_path != save_path:  # new video
                        vid_path = save_path
                        if isinstance(vid_writer, cv2.VideoWriter):
                            vid_writer.release()  # release previous video writer

                        fourcc = 'mp4v'  # output video codec
                        fps = vid_cap.get(cv2.CAP_PROP_FPS)
                        w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                        h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                        vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*fourcc), fps, (w, h))
                    vid_writer.write(im0)

    if save_txt or save_img:
        print('Results saved to %s' % Path(out))
        if platform == 'darwin' and not opt.update:  # MacOS
            os.system('open ' + save_path)

    print('Done. (%.3fs)' % (time.time() - t0))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', nargs='+', type=str, default='yolov4.weights', help='model.pt path(s)')
    parser.add_argument('--source', type=str, default='inference/images', help='source')  # file/folder, 0 for webcam
    parser.add_argument('--output', type=str, default='inference/output', help='output folder')  # output folder
    parser.add_argument('--img-size', type=int, default=640, help='inference size (pixels)')
    parser.add_argument('--conf-thres', type=float, default=0.8, help='object confidence threshold')
    parser.add_argument('--iou-thres', type=float, default=0.5, help='IOU threshold for NMS')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--view-img', action='store_true', help='display results')
    parser.add_argument('--save-txt', action='store_true', help='save results to *.txt')
    parser.add_argument('--classes', nargs='+', type=int, help='filter by class: --class 0, or --class 0 2 3')
    parser.add_argument('--agnostic-nms', action='store_true', help='class-agnostic NMS')
    parser.add_argument('--augment', action='store_true', help='augmented inference')
    parser.add_argument('--update', action='store_true', help='update all models')
    parser.add_argument('--cfg', type=str, default='models/yolov4.cfg', help='*.cfg path')
    parser.add_argument('--names', type=str, default='data/coco.names', help='*.cfg path')
    opt = parser.parse_args()
    print(opt)

    with torch.no_grad():
        if opt.update:  # update all models (to fix SourceChangeWarning)
            for opt.weights in ['']:
                detect()
                strip_optimizer(opt.weights)
        else:
            detect()


usage: ipykernel_launcher.py [-h] [--weights WEIGHTS [WEIGHTS ...]] [--source SOURCE] [--output OUTPUT]
                             [--img-size IMG_SIZE] [--conf-thres CONF_THRES] [--iou-thres IOU_THRES] [--device DEVICE]
                             [--view-img] [--save-txt] [--classes CLASSES [CLASSES ...]] [--agnostic-nms] [--augment]
                             [--update] [--cfg CFG] [--names NAMES]
ipykernel_launcher.py: error: unrecognized arguments: -f C:\Users\p9514\AppData\Roaming\jupyter\runtime\kernel-20a5ce99-e38f-477f-a8c9-3596109a9c6a.json


SystemExit: 2

In [3]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/TLLU4080736.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/TLLU4080736.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('', 0)
出現最多次數的貨號 ('', 0)
384x640 2 persons, Done. (0.273s)
video 1/1 (2/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TL40807345
6
出現最多次數的貨號 ('TL40807345', 1)
出現最多次數的貨號 ('TL40807345', 1)
384x640 2 persons, Done. (0.274s)
video 1/1 (3/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TL40807345', 1)
384x640 1 persons, Done. (0.273s)
video 1/1 (4/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TL40807345', 1)
出現最多次數的貨號 ('TL40807345', 1)
384x640 2 persons, Done. (0.272s)
video 1/1 (5/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現

video 1/1 (55/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done. (0.273s)
video 1/1 (56/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done. (0.272s)
video 1/1 (57/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done. (0.274s)
video 1/1 (58/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
Detected Text: LLU4080734
6
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done. (0.272s)
video 1/1 (59/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
Detected Text: LLU4080734
6
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done. (0.274s)
video 1/1 (60/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 9)
出現最多次數的貨號 ('TLLU408073', 9)
384x640 2 persons, Done.

video 1/1 (113/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 12)
384x640 1 persons, Done. (0.272s)
video 1/1 (114/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.274s)
video 1/1 (115/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.274s)
video 1/1 (116/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.273s)
video 1/1 (117/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.273s)
video 1/1 (118/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.273s)
video 1/1 (119/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 13)
384x640 1 persons, Done. (0.273s)
video 1/1 (120/441) C:\Users\p9514\YOLOv4

video 1/1 (170/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 22)
384x640 1 persons, Done. (0.272s)
video 1/1 (171/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 23)
384x640 1 persons, Done. (0.272s)
video 1/1 (172/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 24)
384x640 1 persons, Done. (0.272s)
video 1/1 (173/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 25)
384x640 1 persons, Done. (0.273s)
video 1/1 (174/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: Detected Text: TLLU408073
6
出現最多次數的貨號 ('TLLU408073', 26)
384x640 1 persons, Done. (0.272s)
video 1/1 (175/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 26)
出現最多次數的貨號 ('TLLU408073', 26)
384x640 2 persons, Done. (0.272s)
video 1/1 (176/441) C:\Users\p951

video 1/1 (278/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.277s)
video 1/1 (279/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (280/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.271s)
video 1/1 (281/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (282/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (283/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (284/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('LLU4080734', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (285/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi:

video 1/1 (338/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
384x640 1 persons, Done. (0.272s)
video 1/1 (339/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (340/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (341/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
384x640 1 persons, Done. (0.273s)
video 1/1 (342/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
384x640 1 persons, Done. (0.274s)
video 1/1 (343/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
出現最多次數的貨號 ('TLLU408073', 44)
384x640 2 persons, Done. (0.272s)
video 1/1 (344/441) C:\Users\p9514\YOLOv4\data\video\TLLU4080736.avi: 出現最多次數的貨號 ('TLLU408073', 44)
出現最多次數的貨號 ('TLLU408073', 44)
384x640 2 persons, Done. (0.272s)
video 1/1 (

In [4]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/TSSU5099400.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/TSSU5099400.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (2/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (3/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (4/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (5/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (6/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('',

video 1/1 (120/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.273s)
video 1/1 (121/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.273s)
video 1/1 (122/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.273s)
video 1/1 (123/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.274s)
video 1/1 (124/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.274s)
video 1/1 (125/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.272s)
video 1/1 (126/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
384x640 1 persons, Done. (0.272s)
video 1/1 (127/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數

出現最多次數的貨號 ('TSSUI50994', 3)
384x640 3 persons, Done. (0.272s)
video 1/1 (180/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 persons, Done. (0.275s)
video 1/1 (181/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 persons, Done. (0.274s)
video 1/1 (182/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 persons, Done. (0.273s)
video 1/1 (183/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 persons, Done. (0.274s)
video 1/1 (184/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 persons, Done. (0.271s)
video 1/1 (185/252) C:\Users\p9514\YOLOv4\data\video\TSSU5099400.avi: 出現最多次數的貨號 ('TSSUI50994', 3)
出現最多次數的貨號 ('TSSUI50994', 3)
384x640 2 pe

In [5]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU2483178.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU2483178.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.271s)
video 1/1 (2/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.272s)
video 1/1 (3/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.272s)
video 1/1 (4/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.271s)
video 1/1 (5/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.271s)
video 1/1 (6/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.273s)
video 1/1 (7/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 384x640 Done. (0.273s)
video 1/1 (8/189) C:\Users\p9514\YOLOv4\data\

video 1/1 (152/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 出現最多次數的貨號 ('WHSU248317', 4)
384x640 1 persons, Done. (0.273s)
video 1/1 (153/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 出現最多次數的貨號 ('WHSU248317', 4)
384x640 1 persons, Done. (0.271s)
video 1/1 (154/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 出現最多次數的貨號 ('WHSU248317', 4)
384x640 1 persons, Done. (0.272s)
video 1/1 (155/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: Detected Text: WHSU248317
8
出現最多次數的貨號 ('WHSU248317', 5)
384x640 1 persons, Done. (0.272s)
video 1/1 (156/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: Detected Text: WHSU248317
8
出現最多次數的貨號 ('WHSU248317', 6)
384x640 1 persons, Done. (0.273s)
video 1/1 (157/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 出現最多次數的貨號 ('WHSU248317', 6)
384x640 1 persons, Done. (0.273s)
video 1/1 (158/189) C:\Users\p9514\YOLOv4\data\video\WHSU2483178.avi: 出現最多次數的貨號 ('WHSU248317', 6)
出現最多次數的貨號 ('WHSU248317', 6)
384x640 2 persons, Done. (

In [6]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU5223791.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU5223791.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.274s)
video 1/1 (2/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.274s)
video 1/1 (3/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.274s)
video 1/1 (4/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (5/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (6/69) C:\Users\p9514\YOLOv4\data\video\WHSU5223791.avi: 出現最多次數的貨號 ('', 0)
38

In [7]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU5368199.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU5368199.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (2/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (3/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.274s)
video 1/1 (4/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (5/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (6/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('',

video 1/1 (117/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 22)
384x640 1 persons, Done. (0.274s)
video 1/1 (118/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 23)
384x640 1 persons, Done. (0.273s)
video 1/1 (119/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 24)
384x640 1 persons, Done. (0.273s)
video 1/1 (120/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 24)
384x640 1 persons, Done. (0.272s)
video 1/1 (121/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 24)
384x640 1 persons, Done. (0.272s)
video 1/1 (122/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 24)
384x640 1 persons, Done. (0.272s)
video 1/1 (123/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 24)
384x640 1 persons, Done. (0.273s)
video 1/1 (12

video 1/1 (248/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.272s)
video 1/1 (249/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.273s)
video 1/1 (250/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.274s)
video 1/1 (251/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.272s)
video 1/1 (252/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.272s)
video 1/1 (253/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.274s)
video 1/1 (254/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 37)
384x640 1 persons, Done. (0.274s)
video 1/1 (255/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi:

video 1/1 (305/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 54)
384x640 1 persons, Done. (0.272s)
video 1/1 (306/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 55)
384x640 1 persons, Done. (0.273s)
video 1/1 (307/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 55)
384x640 1 persons, Done. (0.272s)
video 1/1 (308/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSUGABBLD
9
出現最多次數的貨號 ('WHSU536819', 55)
384x640 1 persons, Done. (0.274s)
video 1/1 (309/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 55)
384x640 1 persons, Done. (0.272s)
video 1/1 (310/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 55)
384x640 1 persons, Done. (0.274s)
video 1/1 (311/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHOUWIDELY
9
出現最多次數的貨號 ('WHSU5368

video 1/1 (357/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 79)
出現最多次數的貨號 ('WHSU536819', 79)
384x640 2 persons, Done. (0.272s)
video 1/1 (358/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 79)
出現最多次數的貨號 ('WHSU536819', 79)
384x640 2 persons, Done. (0.272s)
video 1/1 (359/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 79)
Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 80)
384x640 2 persons, Done. (0.273s)
video 1/1 (360/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 80)
Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 81)
384x640 2 persons, Done. (0.272s)
video 1/1 (361/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 82)
出現最多次數的貨號 ('WHSU536819', 82)
384x640 2 persons, Done. (0.273s)
video 1/1 (362/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected

video 1/1 (462/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 128)
384x640 1 persons, Done. (0.272s)
video 1/1 (463/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 128)
384x640 1 persons, Done. (0.274s)
video 1/1 (464/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 129)
384x640 1 persons, Done. (0.274s)
video 1/1 (465/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: 出現最多次數的貨號 ('WHSU536819', 129)
384x640 1 persons, Done. (0.273s)
video 1/1 (466/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 130)
384x640 1 persons, Done. (0.274s)
video 1/1 (467/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Text: WHSU536819
9
出現最多次數的貨號 ('WHSU536819', 131)
出現最多次數的貨號 ('WHSU536819', 131)
384x640 2 persons, Done. (0.274s)
video 1/1 (468/519) C:\Users\p9514\YOLOv4\data\video\WHSU5368199.avi: Detected Tex

In [8]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU5927851.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU5927851.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.271s)
video 1/1 (2/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.272s)
video 1/1 (3/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.271s)
video 1/1 (4/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.272s)
video 1/1 (5/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.271s)
video 1/1 (6/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.273s)
video 1/1 (7/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 384x640 Done. (0.273s)
video 1/1 (8/326) C:\Users\p9514\YOLOv4\data\

video 1/1 (146/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.274s)
video 1/1 (147/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.273s)
video 1/1 (148/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.273s)
video 1/1 (149/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: Detected Text: WHSUASEFAA
1
出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.273s)
video 1/1 (150/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.272s)
video 1/1 (151/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.273s)
video 1/1 (152/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 2)
384x640 1 persons, Done. (0.274s)
video 1/1 (153/326) C:\Users\p9514\YOLOv4\data\v

video 1/1 (277/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.272s)
video 1/1 (278/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.272s)
video 1/1 (279/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.272s)
video 1/1 (280/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.272s)
video 1/1 (281/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.272s)
video 1/1 (282/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.273s)
video 1/1 (283/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數的貨號 ('WHSU592785', 7)
384x640 1 persons, Done. (0.273s)
video 1/1 (284/326) C:\Users\p9514\YOLOv4\data\video\WHSU5927851.avi: 出現最多次數

In [9]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU6167120.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU6167120.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 1)
384x640 1 persons, Done. (0.272s)
video 1/1 (2/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 1)
384x640 1 persons, Done. (0.274s)
video 1/1 (3/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 2)
384x640 1 persons, Done. (0.272s)
video 1/1 (4/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 2)
384x640 1 persons, Done. (0.273s)
video 1/1 (5/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 

video 1/1 (110/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 43)
出現最多次數的貨號 ('WHSU616712', 43)
384x640 2 persons, Done. (0.272s)
video 1/1 (111/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 44)
出現最多次數的貨號 ('WHSU616712', 44)
384x640 2 persons, Done. (0.274s)
video 1/1 (112/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 44)
出現最多次數的貨號 ('WHSU616712', 44)
Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 45)
384x640 3 persons, Done. (0.275s)
video 1/1 (113/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 45)
384x640 1 persons, Done. (0.272s)
video 1/1 (114/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 45)
384x640 1 persons, Done. (0.272s)
video 1/1 (115/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 46)
384x640 

video 1/1 (166/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.273s)
video 1/1 (167/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.272s)
video 1/1 (168/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.271s)
video 1/1 (169/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.274s)
video 1/1 (170/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.273s)
video 1/1 (171/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.273s)
video 1/1 (172/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 55)
384x640 1 persons, Done. (0.272s)
video 1/1 (173/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi:

video 1/1 (280/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 58)
384x640 1 persons, Done. (0.273s)
video 1/1 (281/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 59)
384x640 1 persons, Done. (0.273s)
video 1/1 (282/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 59)
384x640 1 persons, Done. (0.271s)
video 1/1 (283/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 59)
384x640 1 persons, Done. (0.272s)
video 1/1 (284/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 60)
384x640 1 persons, Done. (0.274s)
video 1/1 (285/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: 出現最多次數的貨號 ('WHSU616712', 60)
384x640 1 persons, Done. (0.273s)
video 1/1 (286/338) C:\Users\p9514\YOLOv4\data\video\WHSU6167120.avi: Detected Text: WHSU616712
0
出現最多次數的貨號 ('WHSU616712', 61)
384x640 1 persons, 

In [10]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU6728690.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU6728690.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.271s)
video 1/1 (2/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (3/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (4/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.273s)
video 1/1 (5/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('', 0)
384x640 1 persons, Done. (0.272s)
video 1/1 (6/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('',

video 1/1 (114/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 26)
384x640 1 persons, Done. (0.272s)
video 1/1 (115/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 27)
384x640 1 persons, Done. (0.274s)
video 1/1 (116/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 28)
384x640 1 persons, Done. (0.274s)
video 1/1 (117/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 29)
384x640 1 persons, Done. (0.272s)
video 1/1 (118/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 30)
384x640 1 persons, Done. (0.273s)
video 1/1 (119/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 31)
384x640 1 persons, Done. (0.271s)
video 1/1 (120/305) C:\Users\p9514

video 1/1 (220/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 100)
384x640 1 persons, Done. (0.272s)
video 1/1 (221/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 101)
384x640 1 persons, Done. (0.272s)
video 1/1 (222/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('WHSU672869', 101)
384x640 1 persons, Done. (0.273s)
video 1/1 (223/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 102)
384x640 1 persons, Done. (0.272s)
video 1/1 (224/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: 出現最多次數的貨號 ('WHSU672869', 102)
384x640 1 persons, Done. (0.272s)
video 1/1 (225/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 103)
384x640 1 persons, Done. (0.272s)
video 1/1 (226/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text:

video 1/1 (266/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 142)
出現最多次數的貨號 ('WHSU672869', 142)
384x640 2 persons, Done. (0.275s)
video 1/1 (267/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 143)
出現最多次數的貨號 ('WHSU672869', 143)
384x640 2 persons, Done. (0.273s)
video 1/1 (268/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 144)
出現最多次數的貨號 ('WHSU672869', 144)
384x640 2 persons, Done. (0.274s)
video 1/1 (269/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 145)
出現最多次數的貨號 ('WHSU672869', 145)
384x640 2 persons, Done. (0.274s)
video 1/1 (270/305) C:\Users\p9514\YOLOv4\data\video\WHSU6728690.avi: Detected Text: WHSU672869
0
出現最多次數的貨號 ('WHSU672869', 146)
出現最多次數的貨號 ('WHSU672869', 146)
384x640 2 persons, Done. (0.272s)
video 1/1 (271/305) C:\Users\p9514\YOLOv

In [11]:
%run detect.py --weights weights/yolo-obj_best.weights --source data/video/WHSU6892256.avi

Namespace(weights=['weights/yolo-obj_best.weights'], source='data/video/WHSU6892256.avi', output='inference/output', img_size=640, conf_thres=0.8, iou_thres=0.5, device='', view_img=False, save_txt=False, classes=None, agnostic_nms=False, augment=False, update=False, cfg='models/yolov4.cfg', names='data/coco.names')
video 1/1 (1/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: Detected Text: WHSU689225
6
出現最多次數的貨號 ('WHSU689225', 1)
出現最多次數的貨號 ('WHSU689225', 1)
384x640 2 persons, Done. (0.274s)
video 1/1 (2/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 1)
384x640 1 persons, Done. (0.273s)
video 1/1 (3/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 1)
384x640 1 persons, Done. (0.274s)
video 1/1 (4/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 1)
384x640 1 persons, Done. (0.273s)
video 1/1 (5/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 1)
384x640 1 

出現最多次數的貨號 ('WHSU689225', 65)
384x640 3 persons, Done. (0.274s)
video 1/1 (104/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 65)
出現最多次數的貨號 ('WHSU689225', 65)
384x640 2 persons, Done. (0.273s)
video 1/1 (105/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: Detected Text: WHSU689225
6
出現最多次數的貨號 ('WHSU689225', 66)
384x640 1 persons, Done. (0.274s)
video 1/1 (106/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 66)
384x640 1 persons, Done. (0.273s)
video 1/1 (107/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: 出現最多次數的貨號 ('WHSU689225', 66)
384x640 1 persons, Done. (0.274s)
video 1/1 (108/108) C:\Users\p9514\YOLOv4\data\video\WHSU6892256.avi: Detected Text: WHSU689225
6
出現最多次數的貨號 ('WHSU689225', 67)
384x640 1 persons, Done. (0.274s)
Results saved to inference\output
Done. (55.218s)
